In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

In [2]:
import re
import string
import emoji

In [3]:
df = pd.read_csv("datasets/final_media_covid.csv")

In [4]:
df.head()

,id,date,username,tweets
0,1607520736603963393,2022-12-26 23:36:24+00:00,Okezone,China Hapus Wajib Karantina Bagi Pelancong Asing di Saat Kasus Covid-19 Melonjak #LengkapCepatBeritanya #BeritaTerkini #International #BeritaInternasional . https://t.co/bXROipUUIP
1,1607514185101103104,2022-12-26 23:10:22+00:00,Okezone,"Covid-19 Kembali Merebak di China, Pencabutan PPKM Perlu Hati-Hati #LengkapCepatBeritanya #BeritaTerkini #Berita #News #BeritaNasional . https://t.co/9VciJm3pkf"
2,1607386138540642305,2022-12-26 14:41:33+00:00,Okezone,"Sebaran 468 Konfirmasi Positif Covid-19 di Indonesia, 6 Provinsi Nihil Kasus #LengkapCepatBeritanya #BeritaTerkini #Berita #News #BeritaNasional . https://t.co/X09MDSUGAW"
3,1607384456578617344,2022-12-26 14:34:52+00:00,Okezone,"'Update' Covid-19 Per Senin 26 Desember 2022: Positif 6.716.592 Orang, Sembuh 6.538.568 dan Meninggal 160.551 #LengkapCepatBeritanya #BeritaTerkini #Berita #News #BeritaNasional . https://t.co/xSXWXUOlhy"
4,1607382275251142659,2022-12-26 14:26:12+00:00,CNN Indonesia,"Positif Covid di RI Tambah 468, Pasien Sembuh 3.212 Orang https://t.co/kqe5YCg525"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36665 entries, 0 to 36664
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        36665 non-null  int64 
 1   date      36665 non-null  object
 2   username  36665 non-null  object
 3   tweets    36665 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


# Check duplicate

In [6]:
# check if it has duplicated value
df.tweets.duplicated().sum()

47

In [7]:
duplicates = df[df.duplicated(subset='tweets')]
duplicates.head()

,id,date,username,tweets
1017,1612460564248399880,2023-01-09 14:45:31+00:00,Tribunnews.com,"Infeksi Covid-19 China Kian Menggila, 90 Persen Warga di Provinsi Henan Dinyatakan Positif\n \nhttps://t.co/MMS3B02kMJ via @tribunnews"
2475,1645598665950777345,2023-04-11 01:24:30+00:00,Harian Kompas,"Pelabuhan Merak memegang kunci sebagai sarana penghubung antarpulau. Pada mudik Lebaran 2023, ada euforia menyambut mudik setelah tahun sebelumnya ada pembatasan karena pandemi Covid-19.\n\nhttps://t.co/zavnz6keCT\n\n #Metropolitan #AdadiKompas"
4260,1497128313013542912,2022-02-25 08:36:19+00:00,TEMPO.CO,"Supaya antibodi terbentuk dengan baik dan tidak timbul risiko tertentu, berikut adalah hal-hal yang tidak dianjurkan bagi seseorang setelah menerima vaksin booster Covid-19. #TempoGaya https://t.co/Gh4vaBuym3"
5259,1500350464424136708,2022-03-06 06:00:00+00:00,CNN Indonesia,Long Covid adalah gejala Covid-19 yang berlangsung dalam waktu lama. Penyintas Omicron pun berisiko mengalami long Covid. Seperti apa gejalanya? #CNNIndonesia \nhttps://t.co/rF8eWwOPmT
6120,1501456501113864195,2022-03-09 07:15:00+00:00,Harian Kompas,"Dua tahun, Covid-19 melanda Indonesia. Penanganan kesehatan dan pemulihan ekonominya tak lepas dari strategi ”gas dan rem”. Dari buku ”Indonesia dan Covid-19: Kerja Keras di Tengah Pandemi”, terekam perjalanannya. #Kesehatan #AdadiKompas\n\nhttps://t.co/3FMEIs4Bff"


In [8]:
# delete duplicate data
# 36665 - 47 = 36618
df.drop_duplicates(subset='tweets', inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        36618 non-null  int64 
 1   date      36618 non-null  object
 2   username  36618 non-null  object
 3   tweets    36618 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


# Select tweets column only

In [10]:
df_tweets = df[['tweets']].copy()
df_tweets.head()

,tweets
0,China Hapus Wajib Karantina Bagi Pelancong Asing di Saat Kasus Covid-19 Melonjak #LengkapCepatBeritanya #BeritaTerkini #International #BeritaInternasional . https://t.co/bXROipUUIP
1,"Covid-19 Kembali Merebak di China, Pencabutan PPKM Perlu Hati-Hati #LengkapCepatBeritanya #BeritaTerkini #Berita #News #BeritaNasional . https://t.co/9VciJm3pkf"
2,"Sebaran 468 Konfirmasi Positif Covid-19 di Indonesia, 6 Provinsi Nihil Kasus #LengkapCepatBeritanya #BeritaTerkini #Berita #News #BeritaNasional . https://t.co/X09MDSUGAW"
3,"'Update' Covid-19 Per Senin 26 Desember 2022: Positif 6.716.592 Orang, Sembuh 6.538.568 dan Meninggal 160.551 #LengkapCepatBeritanya #BeritaTerkini #Berita #News #BeritaNasional . https://t.co/xSXWXUOlhy"
4,"Positif Covid di RI Tambah 468, Pasien Sembuh 3.212 Orang https://t.co/kqe5YCg525"


In [11]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweets  36618 non-null  object
dtypes: object(1)
memory usage: 572.2+ KB


# Cleaning Function

In [12]:
# def remove_non_ascii(text):
#     clean_text = re.sub(r'[^\x00-\x7F]', '', text)
#     return clean_text

def lowercase(text):
    clean_text = text.lower()
    return clean_text

def remove_newline(text):
    clean_text = text.replace('\n', ' ') 
    return clean_text

def remove_hastag(text):
    clean_text = re.sub(r'#\S+', '', text)
    return clean_text

def remove_url(text):
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    clean_text = re.sub(url_pattern, "", text)
    return clean_text

def remove_punctuation(text):
    clean_text = text.translate(str.maketrans("", "", string.punctuation))
    return clean_text

def remove_digits(text):
    clean_text = re.sub(r'\d', '', text)
    return clean_text

def remove_emoji(text):
    clean_text = re.sub(r':[^:]+:', '', emoji.demojize(text))
    return clean_text

# def remove_add_whitespace(text):
#     clean_text = re.sub(r'\s+', ' ', text.strip())
#     return clean_text

# Lowecasing

In [13]:
df_lowercase = df_tweets.copy()

In [14]:
df_lowercase['lowercase'] = df_lowercase['tweets'].apply(lowercase)

In [15]:
df_lowercase.head(100).tail(10)

,tweets,lowercase
90,Data perkembangan pandemi COVID-19 Indonesia hingga Selasa (27/12/2022) : \n\nPositif: 6.717.395 (+803)\nSembuh: 6.540.260 (+1.692)\nMeninggal: 160.560 (+9)\nKasus Aktif: 16.575\n\n#ingatpesanibu #sudahdivaksintetap3m\n#vaksinmelindungikitasemua\n\nhttps://t.co/kxPeWuQVKB,data perkembangan pandemi covid-19 indonesia hingga selasa (27/12/2022) : \n\npositif: 6.717.395 (+803)\nsembuh: 6.540.260 (+1.692)\nmeninggal: 160.560 (+9)\nkasus aktif: 16.575\n\n#ingatpesanibu #sudahdivaksintetap3m\n#vaksinmelindungikitasemua\n\nhttps://t.co/kxpewuqvkb
91,"Indonesia mencatat sebanyak 803 kasus baru COVID-19, Selasa (27/12/2022). Jawa Barat menyumbang kasus terbanyak dengan jumlah 229. https://t.co/wj1aELagX0","indonesia mencatat sebanyak 803 kasus baru covid-19, selasa (27/12/2022). jawa barat menyumbang kasus terbanyak dengan jumlah 229. https://t.co/wj1aelagx0"
92,"Indonesia mencatat 803 kasus baru COVID-19 hari ini, Selasa (27/12/2022). Seiring itu, terdapat kasus sembuh sebanyak 1.692 dan 9 pasien meninggal dunia. https://t.co/XMmBIbjxcM","indonesia mencatat 803 kasus baru covid-19 hari ini, selasa (27/12/2022). seiring itu, terdapat kasus sembuh sebanyak 1.692 dan 9 pasien meninggal dunia. https://t.co/xmmbibjxcm"
93,Badan POM Terbitkan Izin Vaksin Covid-19 Comirnaty Children Usia 6 Bulan hingga 11 Tahun https://t.co/58mb7aihnG,badan pom terbitkan izin vaksin covid-19 comirnaty children usia 6 bulan hingga 11 tahun https://t.co/58mb7aihng
94,Pakar Ikatan Dokter Indonesia menilai kebijakan pencabutan PPKM sudah relevan dengan situasi COVID-19 saat ini. Inikah alasannya? https://t.co/mVB2ZN1f32,pakar ikatan dokter indonesia menilai kebijakan pencabutan ppkm sudah relevan dengan situasi covid-19 saat ini. inikah alasannya? https://t.co/mvb2zn1f32
95,"China tengah kewalahan usai dihantam lonjakan kasus COVID-19, hingga fasilitas kesehatan penuh. Akibatnya, banyak pasien COVID-19 terdampar di luar RS. https://t.co/Tlh6Os0eY6","china tengah kewalahan usai dihantam lonjakan kasus covid-19, hingga fasilitas kesehatan penuh. akibatnya, banyak pasien covid-19 terdampar di luar rs. https://t.co/tlh6os0ey6"
96,Penjualan kembang api terus meningkat seiring dengan pelonggaran kegiatan karena pandemi Covid-19 yang semakin terkendali. Momentum tahun baru dimanfaatkan pedagang kembang api musiman untuk meraup keuntungan maksimal.\n\nhttps://t.co/MuAWAxhGJH\n\n #Metropolitan #AdadiKompas,penjualan kembang api terus meningkat seiring dengan pelonggaran kegiatan karena pandemi covid-19 yang semakin terkendali. momentum tahun baru dimanfaatkan pedagang kembang api musiman untuk meraup keuntungan maksimal.\n\nhttps://t.co/muawaxhgjh\n\n #metropolitan #adadikompas
97,"RSDC Wisma Atlet Kemayoran akan berhenti beroperasi mulai 31 Desember 2022. Saat ini, dari 10 tower yang ada di RSDC Wisma Atlet Kemayoran, tersisa satu tower masih beroperasi untuk menangani pasien Covid-19.\n\nhttps://t.co/1xEXLr9MAU\n\n #Kesehatan #AdadiKompas","rsdc wisma atlet kemayoran akan berhenti beroperasi mulai 31 desember 2022. saat ini, dari 10 tower yang ada di rsdc wisma atlet kemayoran, tersisa satu tower masih beroperasi untuk menangani pasien covid-19.\n\nhttps://t.co/1xexlr9mau\n\n #kesehatan #adadikompas"
98,Dosen UGM: Erick Thohir Sukses Bangun BUMN Tanggap Pandemi Covid-19 https://t.co/iR3Gb2NIt8,dosen ugm: erick thohir sukses bangun bumn tanggap pandemi covid-19 https://t.co/ir3gb2nit8
99,Satgas Penanganan COVID-19 Kepulauan Babel mengawasi penerapan prokes di pusat keramaian.\n\nhttps://t.co/NfZqIWliAQ,satgas penanganan covid-19 kepulauan babel mengawasi penerapan prokes di pusat keramaian.\n\nhttps://t.co/nfzqiwliaq


# Remove newline

In [16]:
df_re_newline = df_lowercase[['lowercase']].copy()

In [17]:
df_re_newline.head()

,lowercase
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak #lengkapcepatberitanya #beritaterkini #international #beritainternasional . https://t.co/bxroipuuip
1,"covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/9vcijm3pkf"
2,"sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/x09mdsugaw"
3,"'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/xsxwxuolhy"
4,"positif covid di ri tambah 468, pasien sembuh 3.212 orang https://t.co/kqe5ycg525"


In [18]:
pattern_newline = r'\n'
matching_rows_newline = df_re_newline[df_re_newline['lowercase'].str.contains(pattern_newline)]
print(matching_rows_newline)

                                                                                                                                                                                                                                                                                     lowercase
28                     ada atau tidak ppkm, sebaiknya kita tetap menjaga kesehatan dan menghindari diri dari tertular penyakit. pengalaman pedih covid-19 beri pelajaran, dampak penyakit ini dapat meluluh-lantakkan berbagai sendi kehidupan. #opini #adadikompas\n\nhttps://t.co/mdklnpb9hj
32                       "asal sero survei kita sudah sampai 90%, ya artinya kita kemungkinan sudah baik. ada apapun, dari manapun seharusnya sudah tidak jadi masalah," ujar jokowi di jakarta, senin (26/12).#mediaindonesia\n#referensibangsa\n#covid-19\n\nsumber: https://t.co/2rpiuvkuso
33                                                               memiliki populasi hingga 65,4 juta jiwa, pemerintah zhejiang mengatakan ba

In [19]:
matching_rows_newline['re_newline'] = matching_rows_newline['lowercase'].apply(remove_newline)
matching_rows_newline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5044 entries, 28 to 36663
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lowercase   5044 non-null   object
 1   re_newline  5044 non-null   object
dtypes: object(2)
memory usage: 118.2+ KB


C:\Users\asus\AppData\Local\Temp\ipykernel_3340\2800929330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows_newline['re_newline'] = matching_rows_newline['lowercase'].apply(remove_newline)


In [20]:
matching_rows_newline.head()

,lowercase,re_newline
28,"ada atau tidak ppkm, sebaiknya kita tetap menjaga kesehatan dan menghindari diri dari tertular penyakit. pengalaman pedih covid-19 beri pelajaran, dampak penyakit ini dapat meluluh-lantakkan berbagai sendi kehidupan. #opini #adadikompas\n\nhttps://t.co/mdklnpb9hj","ada atau tidak ppkm, sebaiknya kita tetap menjaga kesehatan dan menghindari diri dari tertular penyakit. pengalaman pedih covid-19 beri pelajaran, dampak penyakit ini dapat meluluh-lantakkan berbagai sendi kehidupan. #opini #adadikompas https://t.co/mdklnpb9hj"
32,"""asal sero survei kita sudah sampai 90%, ya artinya kita kemungkinan sudah baik. ada apapun, dari manapun seharusnya sudah tidak jadi masalah,"" ujar jokowi di jakarta, senin (26/12).#mediaindonesia\n#referensibangsa\n#covid-19\n\nsumber: https://t.co/2rpiuvkuso","""asal sero survei kita sudah sampai 90%, ya artinya kita kemungkinan sudah baik. ada apapun, dari manapun seharusnya sudah tidak jadi masalah,"" ujar jokowi di jakarta, senin (26/12).#mediaindonesia #referensibangsa #covid-19 sumber: https://t.co/2rpiuvkuso"
33,"memiliki populasi hingga 65,4 juta jiwa, pemerintah zhejiang mengatakan bahwa ada 13.583 pasien covid-19 yang dirawat di rumah sakit provinsi.#mediaindonesia\n#referensibangsa\n#covid-19\n\nsumber: https://t.co/kwrhesvr0f","memiliki populasi hingga 65,4 juta jiwa, pemerintah zhejiang mengatakan bahwa ada 13.583 pasien covid-19 yang dirawat di rumah sakit provinsi.#mediaindonesia #referensibangsa #covid-19 sumber: https://t.co/kwrhesvr0f"
34,"pemeriksaan tes usap pcr dilakukan terhadap seribu pengungsi. dengan temuan 10 orang yang terkonfirmasi covid-19, kata dia, jumlahnya bisa dikatakan relatif sedikit dibanding jumlah pengungsi yang dites usap pcr.\n\n https://t.co/hyzcft7hmt","pemeriksaan tes usap pcr dilakukan terhadap seribu pengungsi. dengan temuan 10 orang yang terkonfirmasi covid-19, kata dia, jumlahnya bisa dikatakan relatif sedikit dibanding jumlah pengungsi yang dites usap pcr. https://t.co/hyzcft7hmt"
35,china memutuskan untuk menghentikan publikasi kasus harian dan kematian akibat covid-19. para pakar kesehatan memperingatkan lonjakan infeksi di china membuka kemungkinan munculnya varian baru covid-19. #internasional #adadikompas\n\nhttps://t.co/xuv7zuqfm7,china memutuskan untuk menghentikan publikasi kasus harian dan kematian akibat covid-19. para pakar kesehatan memperingatkan lonjakan infeksi di china membuka kemungkinan munculnya varian baru covid-19. #internasional #adadikompas https://t.co/xuv7zuqfm7


In [21]:
df_re_newline['re_newline'] = df_re_newline['lowercase'].apply(remove_newline)
df_re_newline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lowercase   36618 non-null  object
 1   re_newline  36618 non-null  object
dtypes: object(2)
memory usage: 858.2+ KB


In [22]:
df_re_newline.head(35).tail(10)

,lowercase,re_newline
25,"dosen universitas gadjah mada (ugm) menjelaskan, hiposmia adalah salah satu gejala penurunan kemampuan membau terhadap sesuatu. https://t.co/xuaqqazvma","dosen universitas gadjah mada (ugm) menjelaskan, hiposmia adalah salah satu gejala penurunan kemampuan membau terhadap sesuatu. https://t.co/xuaqqazvma"
26,pemerintah mulai percaya diri meninggalkan pandemi covid-19. di sisi lain masih sangat mungkin gelombang peningkatan kasus covid-19 kembali terjadi. https://t.co/dr93yubhzs,pemerintah mulai percaya diri meninggalkan pandemi covid-19. di sisi lain masih sangat mungkin gelombang peningkatan kasus covid-19 kembali terjadi. https://t.co/dr93yubhzs
27,"giliran jepang melaporkan kekhawatiran tren kasus covid-19. pasalnya, sejumlah anak-anak dalam kondisi sehat meninggal usai terinfeksi varian omicron. https://t.co/coacvsk0li","giliran jepang melaporkan kekhawatiran tren kasus covid-19. pasalnya, sejumlah anak-anak dalam kondisi sehat meninggal usai terinfeksi varian omicron. https://t.co/coacvsk0li"
28,"ada atau tidak ppkm, sebaiknya kita tetap menjaga kesehatan dan menghindari diri dari tertular penyakit. pengalaman pedih covid-19 beri pelajaran, dampak penyakit ini dapat meluluh-lantakkan berbagai sendi kehidupan. #opini #adadikompas\n\nhttps://t.co/mdklnpb9hj","ada atau tidak ppkm, sebaiknya kita tetap menjaga kesehatan dan menghindari diri dari tertular penyakit. pengalaman pedih covid-19 beri pelajaran, dampak penyakit ini dapat meluluh-lantakkan berbagai sendi kehidupan. #opini #adadikompas https://t.co/mdklnpb9hj"
29,koordinator tim pakar dan juru bicara pemerintah untuk penanganan covid-19 prof wiku adisasmito mengingatkan bahwa kesadaran masyarakat untuk menerapkan protokol kesehatan merupakan kunci utama agar jumlah kasus covid-19 tetap rendah. https://t.co/odump3lipa,koordinator tim pakar dan juru bicara pemerintah untuk penanganan covid-19 prof wiku adisasmito mengingatkan bahwa kesadaran masyarakat untuk menerapkan protokol kesehatan merupakan kunci utama agar jumlah kasus covid-19 tetap rendah. https://t.co/odump3lipa
30,"68,33 juta warga indonesia sudah mendapat vaksinasi covid-19 penguat https://t.co/iyf0hzenjh","68,33 juta warga indonesia sudah mendapat vaksinasi covid-19 penguat https://t.co/iyf0hzenjh"
31,peningkatan kasus covid-19 di tiongkok diharapkan tidak pengaruhi situasi di indonesia https://t.co/gnktskeqmk,peningkatan kasus covid-19 di tiongkok diharapkan tidak pengaruhi situasi di indonesia https://t.co/gnktskeqmk
32,"""asal sero survei kita sudah sampai 90%, ya artinya kita kemungkinan sudah baik. ada apapun, dari manapun seharusnya sudah tidak jadi masalah,"" ujar jokowi di jakarta, senin (26/12).#mediaindonesia\n#referensibangsa\n#covid-19\n\nsumber: https://t.co/2rpiuvkuso","""asal sero survei kita sudah sampai 90%, ya artinya kita kemungkinan sudah baik. ada apapun, dari manapun seharusnya sudah tidak jadi masalah,"" ujar jokowi di jakarta, senin (26/12).#mediaindonesia #referensibangsa #covid-19 sumber: https://t.co/2rpiuvkuso"
33,"memiliki populasi hingga 65,4 juta jiwa, pemerintah zhejiang mengatakan bahwa ada 13.583 pasien covid-19 yang dirawat di rumah sakit provinsi.#mediaindonesia\n#referensibangsa\n#covid-19\n\nsumber: https://t.co/kwrhesvr0f","memiliki populasi hingga 65,4 juta jiwa, pemerintah zhejiang mengatakan bahwa ada 13.583 pasien covid-19 yang dirawat di rumah sakit provinsi.#mediaindonesia #referensibangsa #covid-19 sumber: https://t.co/kwrhesvr0f"
34,"pemeriksaan tes usap pcr dilakukan terhadap seribu pengungsi. dengan temuan 10 orang yang terkonfirmasi covid-19, kata dia, jumlahnya bisa dikatakan relatif sedikit dibanding jumlah pengungsi yang dites usap pcr.\n\n https://t.co/hyzcft7hmt","pemeriksaan tes usap pcr dilakukan terhadap seribu pengungsi. dengan temuan 10 orang yang terkonfirmasi covid-19, kata dia, jumlahnya bisa dikatakan relatif sedikit dibanding jumlah pengungsi yang dites usap pcr. https://t.co/hyzcft7hmt"


# Remove hastag

In [23]:
df_re_hastag = df_re_newline[['re_newline']].copy()

In [24]:
df_re_hastag.head()

,re_newline
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak #lengkapcepatberitanya #beritaterkini #international #beritainternasional . https://t.co/bxroipuuip
1,"covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/9vcijm3pkf"
2,"sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/x09mdsugaw"
3,"'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/xsxwxuolhy"
4,"positif covid di ri tambah 468, pasien sembuh 3.212 orang https://t.co/kqe5ycg525"


In [25]:
df_re_hastag['re_hastag'] = df_re_hastag['re_newline'].apply(remove_hastag)
df_re_hastag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   re_newline  36618 non-null  object
 1   re_hastag   36618 non-null  object
dtypes: object(2)
memory usage: 858.2+ KB


In [26]:
df_re_hastag.head()

,re_newline,re_hastag
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak #lengkapcepatberitanya #beritaterkini #international #beritainternasional . https://t.co/bxroipuuip,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak . https://t.co/bxroipuuip
1,"covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/9vcijm3pkf","covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati . https://t.co/9vcijm3pkf"
2,"sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/x09mdsugaw","sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus . https://t.co/x09mdsugaw"
3,"'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 #lengkapcepatberitanya #beritaterkini #berita #news #beritanasional . https://t.co/xsxwxuolhy","'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 . https://t.co/xsxwxuolhy"
4,"positif covid di ri tambah 468, pasien sembuh 3.212 orang https://t.co/kqe5ycg525","positif covid di ri tambah 468, pasien sembuh 3.212 orang https://t.co/kqe5ycg525"


# Remove Url

In [27]:
df_re_url = df_re_hastag[['re_hastag']].copy()

In [28]:
df_re_url.head()

,re_hastag
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak . https://t.co/bxroipuuip
1,"covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati . https://t.co/9vcijm3pkf"
2,"sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus . https://t.co/x09mdsugaw"
3,"'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 . https://t.co/xsxwxuolhy"
4,"positif covid di ri tambah 468, pasien sembuh 3.212 orang https://t.co/kqe5ycg525"


In [29]:
df_re_url['re_url'] = df_re_url['re_hastag'].apply(remove_url)
df_re_url.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   re_hastag  36618 non-null  object
 1   re_url     36618 non-null  object
dtypes: object(2)
memory usage: 858.2+ KB


In [30]:
df_re_url.head()

,re_hastag,re_url
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak . https://t.co/bxroipuuip,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak .
1,"covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati . https://t.co/9vcijm3pkf","covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati ."
2,"sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus . https://t.co/x09mdsugaw","sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus ."
3,"'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 . https://t.co/xsxwxuolhy","'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 ."
4,"positif covid di ri tambah 468, pasien sembuh 3.212 orang https://t.co/kqe5ycg525","positif covid di ri tambah 468, pasien sembuh 3.212 orang"


# Remove Punctuation

In [31]:
df_re_punct = df_re_url[['re_url']].copy()

In [32]:
df_re_punct.head()

,re_url
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak .
1,"covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati ."
2,"sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus ."
3,"'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 ."
4,"positif covid di ri tambah 468, pasien sembuh 3.212 orang"


In [33]:
df_re_punct['re_punct'] = df_re_punct['re_url'].apply(remove_punctuation)
df_re_punct.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   re_url    36618 non-null  object
 1   re_punct  36618 non-null  object
dtypes: object(2)
memory usage: 858.2+ KB


In [34]:
df_re_punct.head()

,re_url,re_punct
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid-19 melonjak .,china hapus wajib karantina bagi pelancong asing di saat kasus covid19 melonjak
1,"covid-19 kembali merebak di china, pencabutan ppkm perlu hati-hati .",covid19 kembali merebak di china pencabutan ppkm perlu hatihati
2,"sebaran 468 konfirmasi positif covid-19 di indonesia, 6 provinsi nihil kasus .",sebaran 468 konfirmasi positif covid19 di indonesia 6 provinsi nihil kasus
3,"'update' covid-19 per senin 26 desember 2022: positif 6.716.592 orang, sembuh 6.538.568 dan meninggal 160.551 .",update covid19 per senin 26 desember 2022 positif 6716592 orang sembuh 6538568 dan meninggal 160551
4,"positif covid di ri tambah 468, pasien sembuh 3.212 orang",positif covid di ri tambah 468 pasien sembuh 3212 orang


# Remove Digits

In [35]:
df_re_dig = df_re_punct[['re_punct']].copy()

In [36]:
df_re_dig.head()

,re_punct
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid19 melonjak
1,covid19 kembali merebak di china pencabutan ppkm perlu hatihati
2,sebaran 468 konfirmasi positif covid19 di indonesia 6 provinsi nihil kasus
3,update covid19 per senin 26 desember 2022 positif 6716592 orang sembuh 6538568 dan meninggal 160551
4,positif covid di ri tambah 468 pasien sembuh 3212 orang


In [37]:
df_re_dig['re_dig'] = df_re_dig['re_punct'].apply(remove_digits)
df_re_dig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   re_punct  36618 non-null  object
 1   re_dig    36618 non-null  object
dtypes: object(2)
memory usage: 858.2+ KB


In [38]:
df_re_dig.head(10)

,re_punct,re_dig
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid19 melonjak,china hapus wajib karantina bagi pelancong asing di saat kasus covid melonjak
1,covid19 kembali merebak di china pencabutan ppkm perlu hatihati,covid kembali merebak di china pencabutan ppkm perlu hatihati
2,sebaran 468 konfirmasi positif covid19 di indonesia 6 provinsi nihil kasus,sebaran konfirmasi positif covid di indonesia provinsi nihil kasus
3,update covid19 per senin 26 desember 2022 positif 6716592 orang sembuh 6538568 dan meninggal 160551,update covid per senin desember positif orang sembuh dan meninggal
4,positif covid di ri tambah 468 pasien sembuh 3212 orang,positif covid di ri tambah pasien sembuh orang
5,breaking news kasus covid19 bertambah 468 hari ini,breaking news kasus covid bertambah hari ini
6,kenapa covid19 di jepang menggila lagi tembus 177 ribu kasus,kenapa covid di jepang menggila lagi tembus ribu kasus
7,lonjakan kasus covid19 di china bikin pabrik terpaksa tutup,lonjakan kasus covid di china bikin pabrik terpaksa tutup
8,covid19 merebak lagi di china bang saleh ingatkan pemerintah,covid merebak lagi di china bang saleh ingatkan pemerintah
9,kasus covid19 turun rsdc wisma atlet kemayoran segera ditutup,kasus covid turun rsdc wisma atlet kemayoran segera ditutup


# Remove Emoji

In [39]:
df_re_emoji = df_re_dig[['re_dig']].copy()
df_re_emoji.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   re_dig  36618 non-null  object
dtypes: object(1)
memory usage: 572.2+ KB


In [40]:
def has_emoji(text):
    """Check if a string contains emojis."""
    return emoji.emoji_count(text) > 0

# Apply the function to the 'text' column
emoji_rows = df_re_emoji[df_re_emoji['re_dig'].apply(has_emoji)]

In [41]:
print(emoji_rows)

                                                                                                                                                                                                                                                            re_dig
459                                                                                                                      presiden jokowi resmi mencabut ppkm seiring kondisi covid yang makin terkendali apa saja yang berubah cari tahu dalam  berikut yuk👇      
1454                                 detikers kini vaksin covid booster kedua atau dosis keempat sudah bisa didapatkan oleh masyarakat umum selain lansia dan tenaga kesehatan  nah bagaimana ya cara mendapatkan vaksin gratis cari tahu infonya dalam  berikut👇 
1651                          siapa yang lagi cari lokasi vaksin booster kedua di jabodetabek 😃⁣  pemerintah kini memperbolehkan masyarakat umum untuk mendapatkan vaksin covid booster kedua penyuntikan vaksin ini serentak d

In [42]:
emoji_rows['re_emoji'] = emoji_rows['re_dig'].apply(remove_emoji)
emoji_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 459 to 36626
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   re_dig    68 non-null     object
 1   re_emoji  68 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


C:\Users\asus\AppData\Local\Temp\ipykernel_3340\175959332.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emoji_rows['re_emoji'] = emoji_rows['re_dig'].apply(remove_emoji)


In [43]:
emoji_rows.head()

,re_dig,re_emoji
459,presiden jokowi resmi mencabut ppkm seiring kondisi covid yang makin terkendali apa saja yang berubah cari tahu dalam berikut yuk👇,presiden jokowi resmi mencabut ppkm seiring kondisi covid yang makin terkendali apa saja yang berubah cari tahu dalam berikut yuk
1454,detikers kini vaksin covid booster kedua atau dosis keempat sudah bisa didapatkan oleh masyarakat umum selain lansia dan tenaga kesehatan nah bagaimana ya cara mendapatkan vaksin gratis cari tahu infonya dalam berikut👇,detikers kini vaksin covid booster kedua atau dosis keempat sudah bisa didapatkan oleh masyarakat umum selain lansia dan tenaga kesehatan nah bagaimana ya cara mendapatkan vaksin gratis cari tahu infonya dalam berikut
1651,siapa yang lagi cari lokasi vaksin booster kedua di jabodetabek 😃⁣ pemerintah kini memperbolehkan masyarakat umum untuk mendapatkan vaksin covid booster kedua penyuntikan vaksin ini serentak dimulai dari tanggal januari ⁣,siapa yang lagi cari lokasi vaksin booster kedua di jabodetabek ⁣ pemerintah kini memperbolehkan masyarakat umum untuk mendapatkan vaksin covid booster kedua penyuntikan vaksin ini serentak dimulai dari tanggal januari ⁣
1667,siapa yang lagi cari lokasi vaksin booster kedua di jabodetabek 😃⁣ pemerintah kini memperbolehkan masyarakat umum untuk mendapatkan vaksin covid booster kedua penyuntikan vaksin ini serentak dimulai dari tanggal januari ⁣,siapa yang lagi cari lokasi vaksin booster kedua di jabodetabek ⁣ pemerintah kini memperbolehkan masyarakat umum untuk mendapatkan vaksin covid booster kedua penyuntikan vaksin ini serentak dimulai dari tanggal januari ⁣
2123,wah ternyata indonesia menjadi salah satu negara yang berhasil menangani covid✨ keberhasilan indonesia ini menuai pujian serta apresiasi dari who hingga johns hopkins university lho detikers yuk intip ungkapan presiden jokowi terkait hal tersebut,wah ternyata indonesia menjadi salah satu negara yang berhasil menangani covid keberhasilan indonesia ini menuai pujian serta apresiasi dari who hingga johns hopkins university lho detikers yuk intip ungkapan presiden jokowi terkait hal tersebut


In [44]:
df_re_emoji['re_emoji'] = df_re_emoji['re_dig'].apply(remove_emoji)
df_re_emoji.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   re_dig    36618 non-null  object
 1   re_emoji  36618 non-null  object
dtypes: object(2)
memory usage: 858.2+ KB


In [45]:
df_re_emoji.head(1653).tail(5)

,re_dig,re_emoji
1649,berikut ini jumlah kasus covid hingga hari ini untuk provinsi aceh sumut sumbar riau kepri jambi dan bengkulu,berikut ini jumlah kasus covid hingga hari ini untuk provinsi aceh sumut sumbar riau kepri jambi dan bengkulu
1650,china mengeklaim jumlah kematian harian akibat covid turun hampir persen dibandingkan awal januari kantor berita afp melaporkan penurunan ini mungkin pertanda lonjakan kasus covid china mulai mereda lt,china mengeklaim jumlah kematian harian akibat covid turun hampir persen dibandingkan awal januari kantor berita afp melaporkan penurunan ini mungkin pertanda lonjakan kasus covid china mulai mereda lt
1651,siapa yang lagi cari lokasi vaksin booster kedua di jabodetabek 😃⁣ pemerintah kini memperbolehkan masyarakat umum untuk mendapatkan vaksin covid booster kedua penyuntikan vaksin ini serentak dimulai dari tanggal januari ⁣,siapa yang lagi cari lokasi vaksin booster kedua di jabodetabek ⁣ pemerintah kini memperbolehkan masyarakat umum untuk mendapatkan vaksin covid booster kedua penyuntikan vaksin ini serentak dimulai dari tanggal januari ⁣
1652,ada kematian yang disebabkan oleh covid di rumah sakit pada senin turun persen dari januari kata cdc china,ada kematian yang disebabkan oleh covid di rumah sakit pada senin turun persen dari januari kata cdc china
1653,pandemi covid munculkan ide usaha rumahan yang kini menghasilkan puluhan juta rupiah,pandemi covid munculkan ide usaha rumahan yang kini menghasilkan puluhan juta rupiah


# export dataset_preprocessing

In [46]:
df_covid_preprocessing = df_re_emoji[['re_emoji']].copy()

In [47]:
df_covid_preprocessing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   re_emoji  36618 non-null  object
dtypes: object(1)
memory usage: 572.2+ KB


In [48]:
df_covid_preprocessing = df_covid_preprocessing.rename(columns={'re_emoji': 'tweets'})

In [49]:
df_covid_preprocessing.head()

,tweets
0,china hapus wajib karantina bagi pelancong asing di saat kasus covid melonjak
1,covid kembali merebak di china pencabutan ppkm perlu hatihati
2,sebaran konfirmasi positif covid di indonesia provinsi nihil kasus
3,update covid per senin desember positif orang sembuh dan meninggal
4,positif covid di ri tambah pasien sembuh orang


In [50]:
df_covid_preprocessing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36618 entries, 0 to 36664
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweets  36618 non-null  object
dtypes: object(1)
memory usage: 572.2+ KB


In [51]:
#df_covid_preprocessing.to_csv("datasets/covid_preprocessing.csv", encoding="utf-8", index=False)

# END

In [52]:
df_covid_preprocessing.tweets.duplicated().sum()

4308